## Data loading

In [1]:
import sys
sys.path.append('../../_common/database_communicator/')

In [2]:
from db_connector import DBConnector
import pandas as pd

In [3]:
connector = DBConnector()
engine = connector.create_sql_engine()

In [4]:
df = pd.read_sql_query("SELECT * FROM data_staging", con=engine)

## Data cleaning

In [5]:
df['floor'].fillna('brak informacji', inplace=True)
df['status'].fillna('brak informacji', inplace=True)
df['property_type'].fillna('brak informacji', inplace=True)
df['rooms'].fillna(1, inplace=True)
df['year_built'].fillna('brak informacji', inplace=True)
df['property_condition'].fillna('brak informacji', inplace=True)

In [6]:
print(df.duplicated().sum())
df.drop_duplicates(inplace=True)
print(df.duplicated().sum())

84
0


### price column

In [7]:
df['price'] = df['price'].str.replace('zł', '').str \
    .replace(' ', '') \
    .replace(',', '.', regex=True) \
    .replace('Zapytajocenę', None, regex=True) \
    .astype(float)

In [8]:
df = df[df['price'].notna()]
df = df[df['price'] < 5000000.0]

### size column

In [9]:
df['size']= df['size'].str.replace(',', '.').astype(float)

### location column

In [10]:
import pandas as pd
import re

loc_list = ['Grunwald', 'Górczyn', 'Ławica', 'Łazarz', 'Junikowo', 'Jeżyce', 
            'Podolany', 'Sołacz', 'Wilda', 'Dębiec', 'Nowe Miasto', 'Łacina', 
            'Rataje', 'Starołęka Mała', 'Stare Miasto', 'Naramowice', 
            'Piątkowo', 'Winogrady', 'Chartowo']

# Function to find and return matching location from loc_list
def find_location(x):
    for loc in loc_list:
        if bool(re.search(loc, x)):
            return loc
    return 'Poznań'

# Apply the function on 'location' column
df['location'] = df['location'].apply(find_location)

In [11]:
df['location'].value_counts()

location
Stare Miasto    245
Grunwald        228
Nowe Miasto     208
Jeżyce          139
Wilda            58
Poznań           14
Piątkowo          7
Winogrady         6
Podolany          5
Naramowice        4
Rataje            4
Górczyn           2
Łazarz            2
Chartowo          1
Sołacz            1
Junikowo          1
Ławica            1
Name: count, dtype: int64

### floor column

In [12]:
df['floor'] = df['floor'].apply(lambda x: x.split('/')[0] if type(x) == str else x)

In [13]:
import pandas as pd

def extract_numbers(s):
    return ''.join(filter(str.isdigit, s)) if any(map(str.isdigit, s)) else s


df['floor'] = df['floor'].str.replace('parter', '0').str.replace('poddasze', '10').apply(extract_numbers)
df['floor'] = df['floor'].str.replace('zapytaj', 'brak informacji').str.replace('suterena', '-1')

In [14]:
df['floor'].unique()

array(['1', '5', '3', '2', '0', '4', '7', '10', 'brak informacji', '6',
       '-1', '8', '9'], dtype=object)

### status column

In [15]:
df['status'].unique()

array(['wtórny', 'pierwotny', 'brak informacji'], dtype=object)

### property_type column

In [16]:
df['property_type'] = df['property_type'].str.replace('plomba', 'pozostałe').str.replace('bliźniak', 'wolnostojący').str.replace('dom wolnostojący', 'wolnostojący')
df['property_type'].unique()

array(['brak informacji', 'blok', 'apartamentowiec', 'kamienica',
       'pozostałe', 'wolnostojący', 'szeregowiec'], dtype=object)

In [17]:
df['property_type'].value_counts()

property_type
blok               377
brak informacji    276
kamienica          133
apartamentowiec     91
wolnostojący        24
szeregowiec         16
pozostałe            9
Name: count, dtype: int64

### rooms column

In [18]:
df['rooms'].fillna(1, inplace=True)
df['rooms'].unique()

array(['2', '3', '1', '4', '6', '5', 1, '7', '8', '9', '10'], dtype=object)

### year_built column

In [19]:
df['year_built'].unique()

array(['brak informacji', '2023', '1900', '1930', '2003', '1960', '2017',
       '2024', '2012', '1970', '1980', '2009', '1905', '1950', '2020',
       '1962', '2008', '1906', '1912', '1983', '2013', '2010', '2001',
       '2018', '2005', '1902', '1972', '2019', '1990', '2014', '1891',
       '2016', '2015', '1968', '1910', '1996', '2007', '1965', '1985',
       '1923', '2006', '1945', '1955', '1986', '1924', '1963', '2022',
       '1880', '1928', '1978', '2021', '1997', '2011', '2025', '1920',
       '2000', '1999', '1901', '1927', '1929', '1975', '1992', '1969',
       '1908', '1942', '1959', '1961', '1949', '2004', '1911', '1989',
       '1987', '1933', '1976', '1979', '1967', '1964', '1903', '1904',
       '1875', '1938'], dtype=object)

### property_condition column

In [20]:
df['property_condition'] = df['property_condition'].str.replace('zapytaj', 'brak informacji')
df['property_condition'].unique()

array(['do zamieszkania', 'do wykończenia', 'brak informacji',
       'do remontu', 'stan surowy zamknięty'], dtype=object)

## Casting column types

In [21]:
numerical_col = ['size', 'rooms']
categorical_col = ['status', 'property_type', 'floor', 'year_built', 'property_condition', 'location']

for col in numerical_col:
    df[col] = df[col].astype(float)
    
for col in categorical_col:
    df[col] = df[col].astype('category')

In [22]:
df.head()

,url,price,status,size,property_type,rooms,floor,year_built,property_condition,location,desc,image_url
0,https://www.otodom.pl/pl/oferta/nowoczesne-dwu...,630000.0,wtórny,68.00,brak informacji,2.0,1,brak informacji,do zamieszkania,Nowe Miasto,Z przyjemnością przedstawiam ofertę sprzedaży ...,https://ireland.apollo.olxcdn.com/v1/files/eyJ...
1,https://www.otodom.pl/pl/oferta/nowe-3-pokoje-...,552520.0,pierwotny,51.00,blok,3.0,1,2023,do wykończenia,Nowe Miasto,Możliwość kontaktu telefonicznego w godzinach ...,https://ireland.apollo.olxcdn.com/v1/files/eyJ...
2,https://www.olx.pl/d/oferta/mieszkanie-3-pokoj...,649000.0,wtórny,59.00,blok,3.0,1,brak informacji,brak informacji,Naramowice,Zamieszkaj w dzielnicy Poznania blisko natury!...,https://ireland.apollo.olxcdn.com:443/v1/files...
3,https://www.otodom.pl/pl/oferta/wynajety-apart...,679000.0,pierwotny,41.92,apartamentowiec,2.0,5,2023,brak informacji,Nowe Miasto,Oferta bezpośrednio od właściciela – BRAK prow...,https://ireland.apollo.olxcdn.com/v1/files/eyJ...
4,https://www.otodom.pl/pl/oferta/atrakcyjne-mie...,679000.0,wtórny,43.65,kamienica,2.0,3,1900,do zamieszkania,Stare Miasto,* Stylowo urządzone * CHWALISZEWO *\n\nPrzedst...,https://ireland.apollo.olxcdn.com/v1/files/eyJ...


In [23]:
FEAT_COLS = ['status', 'size', 'property_type', 'rooms', 'floor', 'year_built', 'property_condition', 'location']
TARGET_COL = 'price'

X = df[FEAT_COLS]
y = df[TARGET_COL].astype(float)

In [24]:
X.head()

,status,size,property_type,rooms,floor,year_built,property_condition,location
0,wtórny,68.00,brak informacji,2.0,1,brak informacji,do zamieszkania,Nowe Miasto
1,pierwotny,51.00,blok,3.0,1,2023,do wykończenia,Nowe Miasto
2,wtórny,59.00,blok,3.0,1,brak informacji,brak informacji,Naramowice
3,pierwotny,41.92,apartamentowiec,2.0,5,2023,brak informacji,Nowe Miasto
4,wtórny,43.65,kamienica,2.0,3,1900,do zamieszkania,Stare Miasto


In [25]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, shuffle=True)

In [26]:
# X_val.to_csv('X_val.tsv', index=False, sep = '\t')
# y_val.to_csv('y_val.tsv', index=False, sep = '\t')
# X_test.to_csv('X_test.tsv', index=False, sep = '\t')
# y_test.to_csv('y_test.tsv', index=False, sep = '\t')
# X_train.to_csv('X_train.tsv', index=False, sep = '\t')
# y_train.to_csv('y_train.tsv', index=False, sep = '\t')

# Model

In [36]:
import numpy as np

from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV

In [28]:
regressor = XGBRegressor(enable_categorical=True)

In [51]:
## Hyper Parameter Optimization

param_grid = {
    "learning_rate": [0.001, 0.01, 0.1, 0.2, 0.3],
    "n_estimators": [50, 100, 200, 300, 500, 1000, 2000],
    "max_depth": [3, 5, 7, 9, 15, 25, 30],
    "min_child_weight": [1, 3, 5, 7, 10, 15, 20],
    "subsample": [0.5, 0.8, 0.9, 1.0],
    "colsample_bytree": [0.5, 0.8, 0.9, 1.0],
    "gamma": [0, 0.1, 0.2, 0.3, 0.5],
    "scale_pos_weight": [1, 2, 3, 4, 5, 6, 7, 8, 9]
}

In [52]:
# Set up the random search with 4-fold cross validation
random_cv = RandomizedSearchCV(estimator=regressor,
            param_distributions=param_grid,
            cv=4, n_iter=100,
            scoring = 'neg_mean_absolute_error',n_jobs = 8,
            verbose = 3, 
            return_train_score = True,
            random_state=42)

In [53]:
random_cv.fit(X_train,y_train)

Fitting 4 folds for each of 100 candidates, totalling 400 fits
[CV 3/4] END colsample_bytree=0.9, gamma=0, learning_rate=0.3, max_depth=7, min_child_weight=7, n_estimators=200, scale_pos_weight=2, subsample=1.0;, score=(train=-15135.105, test=-154103.600) total time=   0.2s
[CV 1/4] END colsample_bytree=0.9, gamma=0, learning_rate=0.3, max_depth=7, min_child_weight=7, n_estimators=200, scale_pos_weight=2, subsample=1.0;, score=(train=-14527.390, test=-125970.378) total time=   0.3s
[CV 2/4] END colsample_bytree=0.9, gamma=0, learning_rate=0.3, max_depth=7, min_child_weight=7, n_estimators=200, scale_pos_weight=2, subsample=1.0;, score=(train=-17846.356, test=-123703.163) total time=   0.3s
[CV 1/4] END colsample_bytree=0.5, gamma=0.2, learning_rate=0.001, max_depth=15, min_child_weight=15, n_estimators=300, scale_pos_weight=8, subsample=0.5;, score=(train=-251314.318, test=-220475.386) total time=   0.2s
[CV 2/4] END colsample_bytree=0.5, gamma=0.2, learning_rate=0.001, max_depth=15, m

RandomizedSearchCV(cv=4,
                   estimator=XGBRegressor(base_score=None, booster=None,
                                          callbacks=None,
                                          colsample_bylevel=None,
                                          colsample_bynode=None,
                                          colsample_bytree=None, device=None,
                                          early_stopping_rounds=None,
                                          enable_categorical=True,
                                          eval_metric=None, feature_types=None,
                                          gamma=None, grow_policy=None,
                                          importance_type=None,
                                          interaction_constraints=None,
                                          learning_rate=N...
                   param_distributions={'colsample_bytree': [0.5, 0.8, 0.9,
                                                             1.0],
                                        'gamma': [0, 0.1, 0.2, 0.3, 0.5],
                                        'learning_rate': [0.001, 0.01, 0.1, 0.2,
                                                          0.3],
                                        'max_depth': [3, 5, 7, 9, 15, 25, 30],
                                        'min_child_weight': [1, 3, 5, 7, 10, 15,
                                                             20],
                                        'n_estimators': [50, 100, 200, 300, 500,
                                                         1000, 2000],
                                        'scale_pos_weight': [1, 2, 3, 4, 5, 6,
                                                             7, 8, 9],
                                        'subsample': [0.5, 0.8, 0.9, 1.0]},
                   random_state=42, return_train_score=True,
                   scoring='neg_mean_absolute_error', verbose=3)

In [54]:
model = random_cv.best_estimator_
print(random_cv.best_params_)
print(random_cv.best_score_) 

model.feature_importances_

{'subsample': 0.5, 'scale_pos_weight': 2, 'n_estimators': 300, 'min_child_weight': 3, 'max_depth': 30, 'learning_rate': 0.01, 'gamma': 0.2, 'colsample_bytree': 1.0}
-121394.64392905406


array([0.04730931, 0.44505876, 0.06017365, 0.13857387, 0.08460698,
       0.04866053, 0.0846822 , 0.09093471], dtype=float32)

In [56]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
preds = model.predict(X_test)

# Evaluate the model
mae_score = mean_absolute_error(y_test, preds)
print('MAE:', mae_score)

rmse_score = np.sqrt(mean_squared_error(y_test, preds))
print('RMSE:', rmse_score)

r2 = r2_score(y_test, preds)
print('R2:', r2)

MAE: 125237.86097446237
RMSE: 297953.9700542328
R2: 0.6631899238822998
